# MÓDULO 8: INTRODUCCIÓN A LANGGRAPH
## SOLUCIONES - SESIÓN 2: Diseño de Flujos y Multi-Agentes

Este notebook contiene las soluciones completas de los ejercicios de la Sesión 2.

Cada solución incluye:
- Código completo funcional
- Comentarios explicativos de las decisiones clave
- Salidas esperadas

Instalamos las dependencias necesarias.

In [ ]:
# Instalación de dependencias
!pip install -q langgraph langchain langchain-openai grandalf

Importamos los componentes necesarios.

In [ ]:
from typing import TypedDict
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
import random
import string

---
## BLOQUE 1: Ciclos de Refinamiento

### Ejercicio 1: Generador de Tweets con Límite de Caracteres

In [ ]:
# Definición del estado con todos los campos necesarios
class EstadoTweet(TypedDict):
    tema: str
    tweet: str
    longitud_ok: bool
    intentos: int

def nodo_generar_tweet(state: EstadoTweet) -> dict:
    """Genera un tweet sobre el tema.
    Primer intento: tweet largo (>280)
    Intentos posteriores: tweet corto (<280)
    """
    intentos = state["intentos"] + 1
    tema = state["tema"]
    
    # Simulación de generación progresiva
    if intentos == 1:
        tweet = f"Este es un tweet muy extenso sobre {tema}. " * 20  # >280 chars
    else:
        tweet = f"Tweet sobre {tema} con longitud adecuada."
    
    print(f"Intento {intentos}: Tweet generado ({len(tweet)} caracteres)")
    return {"tweet": tweet, "intentos": intentos}

def nodo_validar_longitud(state: EstadoTweet) -> dict:
    """Verifica si el tweet cumple el límite de 280 caracteres."""
    longitud_ok = len(state["tweet"]) <= 280
    print(f"Validador: Longitud {'correcta' if longitud_ok else 'excedida'}")
    return {"longitud_ok": longitud_ok}

def router_tweet(state: EstadoTweet) -> str:
    """Decide si finalizar o reintentar.
    
    Condiciones de salida:
    1. Tweet con longitud correcta
    2. Límite de intentos alcanzado (3)
    """
    if state["longitud_ok"]:
        return "finalizar"
    elif state["intentos"] >= 3:
        return "finalizar"
    else:
        return "reintentar"

def nodo_finalizar(state: EstadoTweet) -> dict:
    """Nodo final que no modifica el estado."""
    return state

Construcción del grafo con arquitectura de ciclo.

In [ ]:
# Construcción del grafo
workflow_tweet = StateGraph(EstadoTweet)

# Añadir nodos
workflow_tweet.add_node("generar", nodo_generar_tweet)
workflow_tweet.add_node("validar", nodo_validar_longitud)
workflow_tweet.add_node("finalizar", nodo_finalizar)

# Configurar punto de entrada
workflow_tweet.set_entry_point("generar")

# Conectar generar → validar (edge incondicional)
workflow_tweet.add_edge("generar", "validar")

# Edge condicional desde validar (CICLO)
workflow_tweet.add_conditional_edges(
    "validar",
    router_tweet,
    {
        "reintentar": "generar",  # Vuelve al generador
        "finalizar": "finalizar"
    }
)

# Conectar finalizar → END
workflow_tweet.add_edge("finalizar", END)

# Compilar
app_tweet = workflow_tweet.compile()

Visualización del grafo.

In [ ]:
# Visualizar estructura del grafo
print(app_tweet.get_graph().draw_ascii())

Ejecución del agente.

In [ ]:
# Estado inicial
estado_inicial = {
    "tema": "Inteligencia Artificial",
    "tweet": "",
    "longitud_ok": False,
    "intentos": 0
}

# Ejecutar el grafo
resultado = app_tweet.invoke(estado_inicial)

# Mostrar resultados
print(f"\n{'='*60}")
print(f"Tweet final: {resultado['tweet']}")
print(f"Longitud: {len(resultado['tweet'])} caracteres")
print(f"Intentos realizados: {resultado['intentos']}")
print(f"{'='*60}")

---
### Ejercicio 2: Validador de Contraseñas con Múltiples Criterios

In [ ]:
# Definición del estado
class EstadoPassword(TypedDict):
    password: str
    es_valida: bool
    intentos: int

def nodo_generar_password(state: EstadoPassword) -> dict:
    """Genera contraseñas aleatorias con progresión de dificultad.
    
    Intentos 1-2: Contraseñas que no cumplen criterios
    Intentos 3+: Contraseñas válidas
    """
    intentos = state["intentos"] + 1
    
    # Simulación de progresión
    if intentos <= 2:
        # Primeros intentos: contraseñas cortas
        password = ''.join(random.choices(string.ascii_letters, k=5))
    else:
        # Intentos posteriores: contraseña válida
        password = ''.join(random.choices(string.ascii_letters + string.digits, k=10))
    
    print(f"Intento {intentos}: Password generada: {password}")
    return {"password": password, "intentos": intentos}

def nodo_validar_password(state: EstadoPassword) -> dict:
    """Valida dos criterios:
    1. Longitud >= 8 caracteres
    2. Contiene al menos un dígito
    """
    password = state["password"]
    
    criterio_longitud = len(password) >= 8
    criterio_numero = any(c.isdigit() for c in password)
    
    es_valida = criterio_longitud and criterio_numero
    
    print(f"Validador: Longitud OK={criterio_longitud}, Número OK={criterio_numero}")
    return {"es_valida": es_valida}

def router_password(state: EstadoPassword) -> str:
    """Router con límite de 5 intentos."""
    if state["es_valida"]:
        return "finalizar"
    elif state["intentos"] >= 5:
        return "finalizar"
    else:
        return "reintentar"

# Construcción del grafo
workflow_password = StateGraph(EstadoPassword)
workflow_password.add_node("generar", nodo_generar_password)
workflow_password.add_node("validar", nodo_validar_password)
workflow_password.add_node("finalizar", lambda state: state)

workflow_password.set_entry_point("generar")
workflow_password.add_edge("generar", "validar")
workflow_password.add_conditional_edges(
    "validar",
    router_password,
    {"reintentar": "generar", "finalizar": "finalizar"}
)
workflow_password.add_edge("finalizar", END)

app_password = workflow_password.compile()

In [ ]:
# Visualización y ejecución
print(app_password.get_graph().draw_ascii())
print("\n")

resultado = app_password.invoke({
    "password": "",
    "es_valida": False,
    "intentos": 0
})

print(f"\n{'='*60}")
print(f"Password final: {resultado['password']}")
print(f"Válida: {resultado['es_valida']}")
print(f"Intentos: {resultado['intentos']}")
print(f"{'='*60}")

---
### Ejercicio 3: Sistema de Refinamiento de Respuestas

In [ ]:
# Definición del estado
class EstadoRespuesta(TypedDict):
    pregunta: str
    keyword: str
    respuesta: str
    contiene_keyword: bool
    intentos: int

def nodo_generar_respuesta(state: EstadoRespuesta) -> dict:
    """Genera respuestas simuladas.
    Primer intento: sin keyword
    Intentos posteriores: con keyword
    """
    intentos = state["intentos"] + 1
    pregunta = state["pregunta"]
    keyword = state["keyword"]
    
    if intentos == 1:
        respuesta = f"Python es una herramienta de programación muy popular."
    else:
        respuesta = f"Python es un {keyword} de programación muy popular."
    
    print(f"Intento {intentos}: {respuesta}")
    return {"respuesta": respuesta, "intentos": intentos}

def nodo_validar_keyword(state: EstadoRespuesta) -> dict:
    """Verifica presencia de keyword (case insensitive)."""
    keyword = state["keyword"].lower()
    respuesta = state["respuesta"].lower()
    
    contiene_keyword = keyword in respuesta
    print(f"Validador: Keyword {'encontrada' if contiene_keyword else 'no encontrada'}")
    return {"contiene_keyword": contiene_keyword}

def router_respuesta(state: EstadoRespuesta) -> str:
    """Router con límite de 3 intentos."""
    if state["contiene_keyword"]:
        return "finalizar"
    elif state["intentos"] >= 3:
        return "finalizar"
    else:
        return "reintentar"

# Construcción del grafo
workflow_respuesta = StateGraph(EstadoRespuesta)
workflow_respuesta.add_node("generar", nodo_generar_respuesta)
workflow_respuesta.add_node("validar", nodo_validar_keyword)
workflow_respuesta.add_node("finalizar", lambda state: state)

workflow_respuesta.set_entry_point("generar")
workflow_respuesta.add_edge("generar", "validar")
workflow_respuesta.add_conditional_edges(
    "validar",
    router_respuesta,
    {"reintentar": "generar", "finalizar": "finalizar"}
)
workflow_respuesta.add_edge("finalizar", END)

app_respuesta = workflow_respuesta.compile()

In [ ]:
# Ejecución
estado_inicial = {
    "pregunta": "¿Qué es Python?",
    "keyword": "lenguaje",
    "respuesta": "",
    "contiene_keyword": False,
    "intentos": 0
}

resultado = app_respuesta.invoke(estado_inicial)

print(f"\n{'='*60}")
print(f"Respuesta final: {resultado['respuesta']}")
print(f"Contiene keyword: {resultado['contiene_keyword']}")
print(f"Intentos: {resultado['intentos']}")
print(f"{'='*60}")

---
## BLOQUE 2: Persistencia y Contexto

### Ejercicio 4: Asistente de Soporte con Memoria de Usuario

In [ ]:
# Definición del estado
class EstadoSoporte(TypedDict):
    nombre_usuario: str
    ticket_id: str
    mensaje: str
    contador_mensajes: int

def nodo_procesar_mensaje(state: EstadoSoporte) -> dict:
    """Procesa mensajes de soporte incrementando el contador.
    
    Este nodo demuestra cómo mantener estado persistente
    a través de múltiples invocaciones.
    """
    contador = state["contador_mensajes"] + 1
    nombre = state["nombre_usuario"]
    ticket = state["ticket_id"]
    mensaje = state["mensaje"]
    
    print(f"Hola {nombre}, este es el mensaje #{contador} del ticket {ticket}")
    print(f"Tu mensaje: {mensaje}")
    
    return {"contador_mensajes": contador}

# Construcción del grafo simple
workflow_soporte = StateGraph(EstadoSoporte)
workflow_soporte.add_node("procesar", nodo_procesar_mensaje)
workflow_soporte.set_entry_point("procesar")
workflow_soporte.add_edge("procesar", END)

# Crear MemorySaver para persistencia
memory_soporte = MemorySaver()

# Compilar CON checkpointer
app_soporte = workflow_soporte.compile(checkpointer=memory_soporte)

Ejecución con el mismo `thread_id` para demostrar persistencia.

In [ ]:
# Configuración con thread_id único
config_soporte = {"configurable": {"thread_id": "ticket_001"}}

# Primera ejecución
print("=== Mensaje 1 ===")
resultado1 = app_soporte.invoke(
    {
        "nombre_usuario": "Ana",
        "ticket_id": "TKT-001",
        "mensaje": "No puedo acceder al sistema",
        "contador_mensajes": 0
    },
    config=config_soporte
)

# Segunda ejecución (debe recordar el contador)
print("\n=== Mensaje 2 ===")
resultado2 = app_soporte.invoke(
    {
        "nombre_usuario": "Ana",
        "ticket_id": "TKT-001",
        "mensaje": "Probé reiniciar el equipo",
        "contador_mensajes": resultado1["contador_mensajes"]
    },
    config=config_soporte
)

# Tercera ejecución
print("\n=== Mensaje 3 ===")
resultado3 = app_soporte.invoke(
    {
        "nombre_usuario": "Ana",
        "ticket_id": "TKT-001",
        "mensaje": "Ya funciona, gracias",
        "contador_mensajes": resultado2["contador_mensajes"]
    },
    config=config_soporte
)

print(f"\n{'='*60}")
print(f"Contador final: {resultado3['contador_mensajes']}")
print(f"{'='*60}")

---
### Ejercicio 5: Carrito de Compras Persistente

In [ ]:
# Definición del estado
class EstadoCarrito(TypedDict):
    productos_totales: int
    ultimo_producto: str
    accion: str  # "añadir" o "ver"

def nodo_gestionar_carrito(state: EstadoCarrito) -> dict:
    """Gestiona acciones del carrito de compras.
    
    Soporta dos acciones:
    - añadir: Incrementa el contador de productos
    - ver: Muestra el estado actual sin modificar
    """
    accion = state["accion"]
    
    if accion == "añadir":
        productos = state["productos_totales"] + 1
        producto = state["ultimo_producto"]
        print(f"Producto añadido: {producto}")
        print(f"Total en carrito: {productos} productos")
        return {"productos_totales": productos}
    elif accion == "ver":
        print(f"Carrito: {state['productos_totales']} productos")
        return state
    else:
        return state

# Construcción del grafo
workflow_carrito = StateGraph(EstadoCarrito)
workflow_carrito.add_node("gestionar", nodo_gestionar_carrito)
workflow_carrito.set_entry_point("gestionar")
workflow_carrito.add_edge("gestionar", END)

# Compilar con persistencia
memory_carrito = MemorySaver()
app_carrito = workflow_carrito.compile(checkpointer=memory_carrito)

In [ ]:
# Ejecución con thread_id consistente
config_carrito = {"configurable": {"thread_id": "usuario_123"}}

# Añadir producto 1
print("=== Añadir Producto 1 ===")
r1 = app_carrito.invoke(
    {"productos_totales": 0, "ultimo_producto": "Laptop", "accion": "añadir"},
    config=config_carrito
)

# Añadir producto 2
print("\n=== Añadir Producto 2 ===")
r2 = app_carrito.invoke(
    {"productos_totales": r1["productos_totales"], "ultimo_producto": "Mouse", "accion": "añadir"},
    config=config_carrito
)

# Ver carrito
print("\n=== Ver Carrito ===")
r3 = app_carrito.invoke(
    {"productos_totales": r2["productos_totales"], "ultimo_producto": "", "accion": "ver"},
    config=config_carrito
)

print(f"\n{'='*60}")
print(f"Productos totales: {r3['productos_totales']}")
print(f"{'='*60}")

---
### Ejercicio 6: Historial de Conversación con Múltiples Usuarios

In [ ]:
# Definición del estado
class EstadoChat(TypedDict):
    usuario: str
    mensaje: str
    contador_interacciones: int

def nodo_chat(state: EstadoChat) -> dict:
    """Procesa mensajes de chat manteniendo contador por usuario.
    
    Demuestra cómo diferentes thread_id mantienen
    estados completamente independientes.
    """
    contador = state["contador_interacciones"] + 1
    usuario = state["usuario"]
    mensaje = state["mensaje"]
    
    print(f"{usuario}: {mensaje} (Interacción #{contador})")
    return {"contador_interacciones": contador}

# Construcción del grafo
workflow_chat = StateGraph(EstadoChat)
workflow_chat.add_node("chat", nodo_chat)
workflow_chat.set_entry_point("chat")
workflow_chat.add_edge("chat", END)

# Compilar con persistencia
memory_chat = MemorySaver()
app_chat = workflow_chat.compile(checkpointer=memory_chat)

In [ ]:
# Ejecución con usuario A
print("=== Usuario A ===")
config_A = {"configurable": {"thread_id": "usuario_A"}}

r_A1 = app_chat.invoke(
    {"usuario": "Alice", "mensaje": "Hola", "contador_interacciones": 0},
    config=config_A
)

r_A2 = app_chat.invoke(
    {"usuario": "Alice", "mensaje": "¿Cómo estás?", "contador_interacciones": r_A1["contador_interacciones"]},
    config=config_A
)

# Ejecución con usuario B (thread_id diferente)
print("\n=== Usuario B ===")
config_B = {"configurable": {"thread_id": "usuario_B"}}

r_B1 = app_chat.invoke(
    {"usuario": "Bob", "mensaje": "Buenos días", "contador_interacciones": 0},
    config=config_B
)

r_B2 = app_chat.invoke(
    {"usuario": "Bob", "mensaje": "¿Qué tal?", "contador_interacciones": r_B1["contador_interacciones"]},
    config=config_B
)

r_B3 = app_chat.invoke(
    {"usuario": "Bob", "mensaje": "Adiós", "contador_interacciones": r_B2["contador_interacciones"]},
    config=config_B
)

# Verificación de independencia
print(f"\n{'='*60}")
print(f"Usuario A: {r_A2['contador_interacciones']} interacciones")
print(f"Usuario B: {r_B3['contador_interacciones']} interacciones")
print(f"Los contadores son independientes: {r_A2['contador_interacciones'] != r_B3['contador_interacciones']}")
print(f"{'='*60}")

---
## BLOQUE 3: Multi-Agente con Aprobación Humana

### Ejercicio 7: Sistema de Traducción con Supervisor

In [ ]:
# Definición del estado
class EstadoTraduccion(TypedDict):
    texto_original: str
    traduccion: str
    keyword_presente: bool
    intentos: int
    aprobado: bool

def nodo_traductor(state: EstadoTraduccion) -> dict:
    """Agente especializado en traducción.
    
    Simulación:
    - Intento 1: Traducción sin keyword "intelligence"
    - Intento 2+: Traducción con keyword
    """
    intentos = state["intentos"] + 1
    
    if intentos == 1:
        traduccion = "This is a text about AI technology."
    else:
        traduccion = "This is a text about artificial intelligence technology."
    
    print(f"Traductor (Intento {intentos}): {traduccion}")
    return {"traduccion": traduccion, "intentos": intentos}

def nodo_revisor(state: EstadoTraduccion) -> dict:
    """Agente especializado en revisión de calidad.
    
    Verifica presencia de la keyword "intelligence".
    """
    keyword_presente = "intelligence" in state["traduccion"].lower()
    print(f"Revisor: Keyword {'encontrada' if keyword_presente else 'no encontrada'}")
    return {"keyword_presente": keyword_presente}

def router_supervisor(state: EstadoTraduccion) -> str:
    """Agente supervisor que decide el flujo.
    
    Patrón Supervisor:
    - Evalúa el trabajo de los agentes especializados
    - Decide si aprobar o solicitar corrección
    - Controla límites de reintentos
    """
    if state["keyword_presente"]:
        print("Supervisor: Traducción aprobada")
        return "publicar"
    elif state["intentos"] >= 3:
        print("Supervisor: Límite alcanzado, aprobando de todas formas")
        return "publicar"
    else:
        print("Supervisor: Solicitando retraducción")
        return "retraducir"

def nodo_publicar(state: EstadoTraduccion) -> dict:
    """Nodo final que marca la traducción como aprobada."""
    print("Sistema: Traducción publicada")
    return {"aprobado": True}

# Construcción del grafo multi-agente
workflow_traduccion = StateGraph(EstadoTraduccion)

# Añadir los 3 agentes especializados + nodo publicador
workflow_traduccion.add_node("traductor", nodo_traductor)
workflow_traduccion.add_node("revisor", nodo_revisor)
workflow_traduccion.add_node("publicar", nodo_publicar)

# Flujo: traductor → revisor
workflow_traduccion.set_entry_point("traductor")
workflow_traduccion.add_edge("traductor", "revisor")

# Edge condicional desde revisor (con CICLO de corrección)
workflow_traduccion.add_conditional_edges(
    "revisor",
    router_supervisor,
    {
        "retraducir": "traductor",  # CICLO: vuelve al traductor
        "publicar": "publicar"
    }
)

workflow_traduccion.add_edge("publicar", END)

# Compilar
app_traduccion = workflow_traduccion.compile()

Visualización y ejecución del sistema multi-agente.

In [ ]:
# Visualizar arquitectura
print(app_traduccion.get_graph().draw_ascii())
print("\n")

# Ejecutar el sistema
estado_inicial = {
    "texto_original": "Texto sobre inteligencia artificial",
    "traduccion": "",
    "keyword_presente": False,
    "intentos": 0,
    "aprobado": False
}

resultado = app_traduccion.invoke(estado_inicial)

print(f"\n{'='*60}")
print(f"Traducción final: {resultado['traduccion']}")
print(f"Keyword presente: {resultado['keyword_presente']}")
print(f"Aprobado: {resultado['aprobado']}")
print(f"Intentos: {resultado['intentos']}")
print(f"{'='*60}")

---
### Ejercicio 8: Sistema de Publicación con Pausa Humana

In [ ]:
# Reutilizar los nodos del ejercicio 7
# (nodo_traductor, nodo_revisor, router_supervisor, nodo_publicar ya están definidos)

# Reconstruir el grafo con las mismas conexiones
workflow_traduccion_pausa = StateGraph(EstadoTraduccion)
workflow_traduccion_pausa.add_node("traductor", nodo_traductor)
workflow_traduccion_pausa.add_node("revisor", nodo_revisor)
workflow_traduccion_pausa.add_node("publicar", nodo_publicar)

workflow_traduccion_pausa.set_entry_point("traductor")
workflow_traduccion_pausa.add_edge("traductor", "revisor")
workflow_traduccion_pausa.add_conditional_edges(
    "revisor",
    router_supervisor,
    {"retraducir": "traductor", "publicar": "publicar"}
)
workflow_traduccion_pausa.add_edge("publicar", END)

# Crear MemorySaver
memory_traduccion = MemorySaver()

# Compilar CON checkpointer e interrupt_before
app_traduccion_pausa = workflow_traduccion_pausa.compile(
    checkpointer=memory_traduccion,
    interrupt_before=["publicar"]  # PAUSA antes de publicar
)

Ejecución demostrando la pausa para aprobación humana.

In [ ]:
# Configuración con thread_id
config_trad = {"configurable": {"thread_id": "traduccion_001"}}

print("=== Ejecución hasta el punto de pausa ===")
resultado = app_traduccion_pausa.invoke(estado_inicial, config=config_trad)

print(f"\n{'='*60}")
print("Estado antes de publicar:")
print(f"Traducción: {resultado['traduccion']}")
print(f"Keyword presente: {resultado['keyword_presente']}")
print(f"Aprobado: {resultado['aprobado']}")  # Debe ser False
print(f"\n⏸ Sistema pausado - Esperando aprobación humana")
print(f"Para continuar: app_traduccion_pausa.invoke(None, config=config_trad)")
print(f"{'='*60}")

# Nota: En producción, aquí esperarías input del usuario
# Para continuar después de la aprobación:
# resultado_final = app_traduccion_pausa.invoke(None, config=config_trad)

---
### Ejercicio 9: Equipo de Revisión de Documentos

In [ ]:
# Definición del estado
class EstadoDocumento(TypedDict):
    contenido: str
    gramatica_ok: bool
    formateado: bool
    intentos: int
    aprobado: bool

def nodo_escritor(state: EstadoDocumento) -> dict:
    """Agente escritor: Genera el documento."""
    intentos = state["intentos"] + 1
    contenido = f"Documento versión {intentos}"
    print(f"Escritor: {contenido}")
    return {"contenido": contenido, "intentos": intentos}

def nodo_editor(state: EstadoDocumento) -> dict:
    """Agente editor: Verifica gramática.
    
    Simulación: Aprueba en el 2do intento.
    """
    gramatica_ok = state["intentos"] >= 2
    print(f"Editor: Gramática {'correcta' if gramatica_ok else 'incorrecta'}")
    return {"gramatica_ok": gramatica_ok}

def router_gramatica(state: EstadoDocumento) -> str:
    """Router: Decide si el documento pasa a formateo o vuelve al escritor."""
    if state["gramatica_ok"]:
        return "formatear"
    else:
        return "reescribir"

def nodo_formateador(state: EstadoDocumento) -> dict:
    """Agente formateador: Añade formato al documento."""
    contenido = "[FORMATEADO] " + state["contenido"]
    print(f"Formateador: {contenido}")
    return {"contenido": contenido, "formateado": True}

def nodo_aprobar_documento(state: EstadoDocumento) -> dict:
    """Nodo de aprobación final."""
    print("Documento aprobado y publicado")
    return {"aprobado": True}

# Construcción del grafo con ciclo de corrección
workflow_documento = StateGraph(EstadoDocumento)

# Añadir los 4 agentes
workflow_documento.add_node("escritor", nodo_escritor)
workflow_documento.add_node("editor", nodo_editor)
workflow_documento.add_node("formateador", nodo_formateador)
workflow_documento.add_node("aprobar", nodo_aprobar_documento)

# Configurar flujo
workflow_documento.set_entry_point("escritor")
workflow_documento.add_edge("escritor", "editor")

# Edge condicional con ciclo de corrección
workflow_documento.add_conditional_edges(
    "editor",
    router_gramatica,
    {
        "reescribir": "escritor",  # CICLO
        "formatear": "formateador"
    }
)

workflow_documento.add_edge("formateador", "aprobar")
workflow_documento.add_edge("aprobar", END)

# Compilar con PAUSA antes de aprobar
app_documento = workflow_documento.compile(
    checkpointer=MemorySaver(),
    interrupt_before=["aprobar"]
)

Visualización y ejecución con pausa.

In [ ]:
# Visualizar
print(app_documento.get_graph().draw_ascii())
print("\n")

# Ejecutar hasta el punto de pausa
config_doc = {"configurable": {"thread_id": "doc_001"}}
estado_inicial_doc = {
    "contenido": "",
    "gramatica_ok": False,
    "formateado": False,
    "intentos": 0,
    "aprobado": False
}

resultado = app_documento.invoke(estado_inicial_doc, config=config_doc)

print(f"\n{'='*60}")
print(f"Contenido antes de aprobar: {resultado['contenido']}")
print(f"Formateado: {resultado['formateado']}")
print(f"Aprobado: {resultado['aprobado']}")  # Debe ser False
print(f"\n⏸ Sistema pausado - Esperando aprobación final")
print(f"{'='*60}")

---
### Ejercicio 10: Sistema Completo de Publicación de Artículos

In [ ]:
# Definición del estado completo
class EstadoArticulo(TypedDict):
    titulo: str
    contenido: str
    keyword: str
    seo_ok: bool
    keyword_ok: bool
    intentos: int
    publicado: bool

def nodo_redactor(state: EstadoArticulo) -> dict:
    """Agente redactor: Genera contenido del artículo.
    
    Progresión simulada:
    - Intento 1: Contenido corto (<100 chars)
    - Intento 2: Contenido largo sin keyword
    - Intento 3+: Contenido largo con keyword
    """
    intentos = state["intentos"] + 1
    titulo = state["titulo"]
    keyword = state["keyword"]
    
    if intentos == 1:
        # Falla validación SEO (muy corto)
        contenido = f"Breve artículo sobre {titulo}."
    elif intentos == 2:
        # Pasa SEO pero falla keyword
        contenido = f"Este es un artículo extenso sobre {titulo}. " * 10
    else:
        # Pasa ambas validaciones
        contenido = f"Este es un artículo extenso sobre {titulo} que incluye {keyword}. " * 10
    
    print(f"Redactor (Intento {intentos}): {len(contenido)} caracteres")
    return {"contenido": contenido, "intentos": intentos}

def nodo_verificador_seo(state: EstadoArticulo) -> dict:
    """Agente SEO: Verifica longitud mínima."""
    seo_ok = len(state["contenido"]) > 100
    print(f"Verificador SEO: {'Aprobado' if seo_ok else 'Rechazado'} (longitud: {len(state['contenido'])})")
    return {"seo_ok": seo_ok}

def router_seo(state: EstadoArticulo) -> str:
    """Router SEO: Decide si continuar o reintentar."""
    if state["seo_ok"]:
        return "continuar_corrector"
    elif state["intentos"] >= 5:
        return "forzar_publicacion"
    else:
        return "reescribir"

def nodo_corrector(state: EstadoArticulo) -> dict:
    """Agente corrector: Verifica presencia de keyword."""
    keyword_ok = state["keyword"].lower() in state["contenido"].lower()
    print(f"Corrector: Keyword {'encontrada' if keyword_ok else 'no encontrada'}")
    return {"keyword_ok": keyword_ok}

def router_corrector(state: EstadoArticulo) -> str:
    """Router corrector: Decide publicación o corrección."""
    if state["keyword_ok"]:
        return "publicar"
    elif state["intentos"] >= 5:
        return "publicar"
    else:
        return "reescribir"

def nodo_publicador(state: EstadoArticulo) -> dict:
    """Nodo final: Marca artículo como publicado."""
    print("Publicador: Artículo publicado exitosamente")
    return {"publicado": True}

def nodo_forzar_publicacion(state: EstadoArticulo) -> dict:
    """Nodo de emergencia: Publicación forzada por límite."""
    print("Sistema: Publicación forzada por límite de intentos")
    return {"publicado": True}

# Construcción del grafo complejo con 2 ciclos
workflow_articulo = StateGraph(EstadoArticulo)

# Añadir todos los nodos
workflow_articulo.add_node("redactor", nodo_redactor)
workflow_articulo.add_node("verificador_seo", nodo_verificador_seo)
workflow_articulo.add_node("corrector", nodo_corrector)
workflow_articulo.add_node("publicador", nodo_publicador)
workflow_articulo.add_node("forzar_publicacion", nodo_forzar_publicacion)

# Configurar flujo con PRIMER CICLO (validación SEO)
workflow_articulo.set_entry_point("redactor")
workflow_articulo.add_edge("redactor", "verificador_seo")

# Edge condicional con CICLO SEO
workflow_articulo.add_conditional_edges(
    "verificador_seo",
    router_seo,
    {
        "reescribir": "redactor",  # CICLO 1
        "continuar_corrector": "corrector",
        "forzar_publicacion": "forzar_publicacion"
    }
)

# SEGUNDO CICLO (validación keyword)
workflow_articulo.add_conditional_edges(
    "corrector",
    router_corrector,
    {
        "reescribir": "redactor",  # CICLO 2
        "publicar": "publicador"
    }
)

# Conectar nodos finales
workflow_articulo.add_edge("publicador", END)
workflow_articulo.add_edge("forzar_publicacion", END)

# Compilar con persistencia + pausa
app_articulo = workflow_articulo.compile(
    checkpointer=MemorySaver(),
    interrupt_before=["publicador"]
)

Visualización y ejecución del sistema completo.

In [ ]:
# Visualizar arquitectura compleja
print("Arquitectura del sistema (2 ciclos de validación):")
print(app_articulo.get_graph().draw_ascii())
print("\n")

# Ejecutar sistema completo
config_art = {"configurable": {"thread_id": "articulo_001"}}
estado_inicial_art = {
    "titulo": "Inteligencia Artificial en 2026",
    "contenido": "",
    "keyword": "machine learning",
    "seo_ok": False,
    "keyword_ok": False,
    "intentos": 0,
    "publicado": False
}

resultado = app_articulo.invoke(estado_inicial_art, config=config_art)

print(f"\n{'='*60}")
print("Estado antes de publicar:")
print(f"Contenido: {resultado['contenido'][:100]}...")
print(f"Longitud total: {len(resultado['contenido'])} caracteres")
print(f"SEO OK: {resultado['seo_ok']}")
print(f"Keyword OK: {resultado['keyword_ok']}")
print(f"Intentos: {resultado['intentos']}")
print(f"Publicado: {resultado['publicado']}")  # Debe ser False
print(f"\n⏸ Sistema pausado - Esperando aprobación final")
print(f"{'='*60}")